In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today().strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제가 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')

#### 종합쇼핑몰 납품상세내역

In [7]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [8]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [9]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [10]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

In [11]:
# 수요기관 필터링
dist_nm = ('|').join(all_shop_df_fin[all_shop_df_fin['수요기관지역명'].str.split(' ').str[1].notnull()]['수요기관지역명'].str.split(' ').str[1])

In [12]:
# 사업명 split 키워드 필터링
mapping_keywd_all = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' '))) ## 전체 키워드
mapping_keywd_all_filter_cnt = pd.Series(Counter(mapping_keywd_all))

mapping_keywd3 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:3])) ## split 3개 키워드
mapping_keywd3_filter_cnt = pd.Series(Counter(mapping_keywd3))

mapping_keywd2 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:2])) ## split 2개 키워드
mapping_keywd2_filter_cnt = pd.Series(Counter(mapping_keywd2))

mapping_keywd1 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:1])) ## split 1개 키워드
mapping_keywd1_filter_cnt = pd.Series(Counter(mapping_keywd1))

mapping_keywd_all_filter_cnt_nm = ('|').join(mapping_keywd_all_filter_cnt[mapping_keywd_all_filter_cnt <= mapping_keywd_all_filter_cnt.mean()].keys())
mapping_keywd3_filter_cnt_nm = ('|').join(mapping_keywd3_filter_cnt[mapping_keywd3_filter_cnt <= mapping_keywd3_filter_cnt.mean()].keys())
mapping_keywd2_filter_cnt_nm = ('|').join(mapping_keywd2_filter_cnt[mapping_keywd2_filter_cnt <= mapping_keywd2_filter_cnt.mean()].keys())
mapping_keywd1_filter_cnt_nm = ('|').join(mapping_keywd1_filter_cnt[mapping_keywd1_filter_cnt <= mapping_keywd1_filter_cnt.mean()].keys())

In [13]:
# 전체 키워드 데이터셋
mapping_keywd_df = pd.DataFrame(Counter(mapping_keywd_all).items(),columns=['키워드','빈도수'])
mapping_keywd_df = mapping_keywd_df[mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
8,인조잔디,346
5,관급자재,223
31,구입,176
30,관급자재인조잔디,169
34,구매,105
...,...,...
585,미래관,1
584,고덕생활축구장,1
582,조성확장공사,1
581,회천,1


In [14]:
# 종합쇼핑몰 기준 중요키워드 추출
print('1등급 키워드 : ', list(mapping_keywd_df[mapping_keywd_df['빈도수'] >= 50].sort_values('빈도수',ascending=False)['키워드']))
print('2등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 50) & (mapping_keywd_df['빈도수'] >= 40)].sort_values('빈도수',ascending=False)['키워드']))
print('3등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 40) & (mapping_keywd_df['빈도수'] >= 30)].sort_values('빈도수',ascending=False)['키워드']))
print('4등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 30) & (mapping_keywd_df['빈도수'] >= 20)].sort_values('빈도수',ascending=False)['키워드']))
print('5등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 20) & (mapping_keywd_df['빈도수'] >= 10)].sort_values('빈도수',ascending=False)['키워드']))
print('기타 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)].sort_values('빈도수',ascending=False)['키워드']))

1등급 키워드 :  ['인조잔디', '관급자재', '구입', '관급자재인조잔디', '구매', '운동장', '테니스장', '조성공사', '게이트볼장', '공사', '체육시설']
2등급 키워드 :  []
3등급 키워드 :  ['정비공사', '조성사업', '교체공사', '관급인조잔디', '설치', '축구장']
4등급 키워드 :  ['풋살장', '개선공사', '조성', '족구장', '파크골프장', '구입인조잔디']
5등급 키워드 :  ['교체', '정비사업', '관급', '환경개선공사', '다목적구장', '조성공사인조잔디', '보수공사', '교체공사인조잔디', '설치공사', '환경개선', '연병장', '다목적', '부대', '정비공사인조잔디', '체육공원']
기타 키워드 :  ['시설개선사업', '생활체육시설', '시설', '조달', '시설개선공사', '개보수공사', '개보수', '조달구매', '신축', '생활체육공원', '개선', '교체사업', '노후시설', '따른', '정비', '시설개선', '추가', '개소', '어린이공원', '위한', '요청', '체육', '증설공사', '실내게이트볼장', '관급자재조경인조잔디', '당진', '개선사업', '김영훈', '공공하수처리시설', '리모델링', '설치공사인조잔디', '조성사업인조잔디', '관련', '사업', '트랙조성', '인조잔디운동장', '본부', '물품', '보수', '일원', '구매인조잔디', '하부', '근린공원', '학교운동장', '관급자재인조잔디구입', '여단', '공사인조잔디', '시행', '환경개선인조잔디', '배드민턴장', '비가림시설', '그라운드골프장', '환경개선사업', '운동공간', '야구장', '재조성', '조달구입', '잔디', '시설물', '옥외', '스포츠파크', '교체관급인조잔디', '구매설치', '수해복구공사', '수지아르피아', '트랙', '환경친화적', '확장', '본원', '트랙보수', '사단', '군단', '기타시설공사', '해운대수목원', '설치인조잔디', '조성단계차', 

In [15]:
# 빈도수 키워드
freq_kwd = ('|').join(list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)]['키워드']))

#### 지자체 세부사업별 예산서

In [434]:
# 빅쿼리에서 불러오기
budget_df_today = get_dataframe_from_bigquery('budget','budget_df_0' + today,bigquerykey_path)
budget_df_ytday = get_dataframe_from_bigquery('budget','budget_df_0' + ytday,bigquerykey_path)

In [435]:
# 필요키워드1
need_kwd1 = '인조잔디|학교|초등학교|중학교|고등학교|대학교|운동장|다목적|종합운동장|공원|체육공원|체육시설|체육센터|스포츠센터|관급|연병장|족구장|축구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장' 

# 필요키워드2
need_kwd2 = '개선|교체|공사|구매|구입|설치|정비|조성'

# 제외키워드
except_kwd = '경관|조명|펜스|복지|교육|학습|보건|행사|예방|안전|치안|창출|활성화|유아|검진|조명|오염|예술|급식|보상|주차장|개방|CCTV|미술|교실|관리|운영|자전거|울타리|버스|맨홀|노점|노상|하천|하수도|광고|약수터|전통시장|계단|정화조|도로확장|골프장|입학|도로개설|소프트웨어|가속기|진로진학|인터넷|음악|취업|역량|억제제|살포|급수관|하수관로|요양기관|도로방음벽|가로등|도로개설|보행자도로|진입도로|철거'

In [436]:
# 키워드 필터링
budget_df_today = budget_df_today[budget_df_today['세부사업명'].str.contains(need_kwd1) & ~budget_df_today['세부사업명'].str.contains(except_kwd)].reset_index(drop=True)
budget_df_ytday = budget_df_ytday[budget_df_ytday['세부사업명'].str.contains(need_kwd1) & ~budget_df_ytday['세부사업명'].str.contains(except_kwd)].reset_index(drop=True)

In [437]:
# 종료 사업
budget_df_delete_temp1 = budget_df_ytday[budget_df_ytday['세부사업코드'].isin(set(budget_df_ytday['세부사업코드']) - set(budget_df_today['세부사업코드']))].reset_index(drop=True)
budget_df_delete_temp2 = budget_df_ytday[budget_df_ytday['세부사업명'].isin(set(budget_df_ytday['세부사업명']) - set(budget_df_today['세부사업명']))].reset_index(drop=True)
budget_df_delete = pd.concat([budget_df_delete_temp1,budget_df_delete_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_delete

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드


In [438]:
# 새로 추가된 사업
budget_df_new_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(budget_df_ytday['세부사업코드'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new_temp2 = budget_df_today[~budget_df_today['세부사업명'].isin(budget_df_ytday['세부사업명'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new = pd.concat([budget_df_new_temp1,budget_df_new_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_new

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드
0,2024,1100000,서울,1100000,서울본청,학교용지부담금특별회계,6110000202030354,학교용지 매입비 부담,20240704,1987525000,0,1987525000,교육,유아및초중등교육,6110000
1,2024,1100000,서울,1100000,서울본청,일반회계,6110000200830744,한강공원 시민이용 홍보,20240704,1227083500,357708176,1217090000,환경,환경보호일반,6110000
2,2024,1100000,서울,1100000,서울본청,일반회계,6110000200832053,한강공원 기초질서 유지,20240704,507269760,276712300,476846000,환경,환경보호일반,6110000
3,2024,1100000,서울,1100000,서울본청,일반회계,611000020133006A,한강공원 나들목 신설 및 개선,20240704,6339972620,565495620,2587720000,환경,환경보호일반,6110000
4,2024,1100000,서울,1100000,서울본청,도시개발특별회계,6110000202230317,서리풀공원 교량형 보행연결로 설치,20240704,1081690630,1060730350,0,환경,상하수도ㆍ수질,6110000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3366,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430051,장기미집행 도시계획시설(평대공원) 조성사업,20240704,180000000,75600000,180000000,국토및지역개발,지역및도시,6500000
3367,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430052,장기미집행 도시계획시설(두모공원) 조성사업,20240704,60000000,36101100,60000000,국토및지역개발,지역및도시,6500000
3368,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430053,장기미집행 도시계획시설(고산공원) 조성사업,20240704,60000000,29500000,60000000,국토및지역개발,지역및도시,6500000
3369,2024,4900000,제주,4900000,제주본청,일반회계,6500000202430080,장기미집행 도시계획시설(새섬공원) 조성사업,20240704,200000000,182100670,200000000,국토및지역개발,지역및도시,6500000


In [439]:
# 사업명 전처리
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명'].apply(clean_text)
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.strip()

In [440]:
# 해당 지역 추출
filtered_budget_df_temp1 = budget_df_today[budget_df_today['자치단체명'].str.contains(dist_nm)].reset_index(drop=True)
filtered_budget_df_temp2 = budget_df_today[budget_df_today['자치단체명'].str.contains('본청')].reset_index(drop=True)

filtered_budget_df_today = pd.concat([filtered_budget_df_temp1,filtered_budget_df_temp2],axis=0).reset_index(drop=True)

In [441]:
# split 키워드 필터링
filtered_budget_df_today_temp1 = filtered_budget_df_today[filtered_budget_df_today['세부사업명_re'].str.contains(mapping_keywd_all_filter_cnt_nm)] ## 전체 키워드
filtered_budget_df_today_temp1 = filtered_budget_df_today_temp1[filtered_budget_df_today_temp1['세부사업명_re'].str.contains(mapping_keywd3_filter_cnt_nm)] ## split 3개 키워드
filtered_budget_df_today_temp1 = filtered_budget_df_today_temp1[filtered_budget_df_today_temp1['세부사업명_re'].str.contains(mapping_keywd2_filter_cnt_nm)] ## split 2개 키워드
filtered_budget_df_today_temp1 = filtered_budget_df_today_temp1[filtered_budget_df_today_temp1['세부사업명_re'].str.contains(mapping_keywd1_filter_cnt_nm)] ## split 1개 키워드

filtered_budget_df_today_temp2 = filtered_budget_df_today[filtered_budget_df_today['세부사업명'].str.contains(mapping_keywd_all_filter_cnt_nm)] ## 전체 키워드
filtered_budget_df_today_temp2 = filtered_budget_df_today_temp2[filtered_budget_df_today_temp2['세부사업명'].str.contains(mapping_keywd3_filter_cnt_nm)] ## split 3개 키워드
filtered_budget_df_today_temp2 = filtered_budget_df_today_temp2[filtered_budget_df_today_temp2['세부사업명'].str.contains(mapping_keywd2_filter_cnt_nm)] ## split 2개 키워드
filtered_budget_df_today_temp2 = filtered_budget_df_today_temp2[filtered_budget_df_today_temp2['세부사업명'].str.contains(mapping_keywd1_filter_cnt_nm)] ## split 1개 키워드

filtered_budget_df_today_fin = pd.concat([filtered_budget_df_today_temp1,filtered_budget_df_today_temp2],axis=0).drop_duplicates().reset_index(drop=True)

In [442]:
# 종합쇼핑몰 기준 필터링
bid_cd_split_today = list(budget_df_today[budget_df_today['세부사업코드'].isin(filtered_budget_df_today_fin['세부사업코드'])]['세부사업코드']) # split 키워드 필터링
bid_cd_freq_today = list(budget_df_today[budget_df_today['세부사업명_re'].str.contains(freq_kwd)]['세부사업코드']) + list(budget_df_today[budget_df_today['세부사업명'].str.contains(freq_kwd)]['세부사업코드'])# 빈도수 키워드 필터링

In [443]:
# 종합쇼핑몰 진행건 제외 & 지출액0(미진행건) 추가
budget_df_today_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(set(bid_cd_split_today + bid_cd_freq_today))]
budget_df_today_temp2 = budget_df_today[budget_df_today['세부사업코드'].isin(set(bid_cd_split_today + bid_cd_freq_today))]
budget_df_today_temp2 = budget_df_today_temp2[budget_df_today_temp2['지출액']==0].reset_index(drop=True)
budget_df_today_fin = pd.concat([budget_df_today_temp1, budget_df_today_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [444]:
# 중요 사업 체크
budget_df_today_final = pd.concat([budget_df_today_fin[budget_df_today_fin['세부사업명'].str.contains('인조잔디')],
                                   budget_df_today_fin[~budget_df_today_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

# budget_df_ytday_final = pd.concat([budget_df_ytday_fin[budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')],
#                                    budget_df_ytday_fin[~budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

In [445]:
# 최종데이터셋
budget_df_today_final = budget_df_today_final[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_delete = budget_df_delete[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_new = budget_df_new[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)

In [446]:
budget_df_today_final

,회계연도,집행일자,지역명,자치단체명,회계구분명,세부사업명,예산현액,지출액,편성액,분야명,부문명
0,2024,20240704,서울,서울구로구,일반회계,안양천 족구장 인조잔디 조성,30508440,0,0,문화및관광,체육
1,2024,20240704,대구,대구북구,일반회계,가로수 보호 인조잔디 매트 설치,22400000,0,22400000,국토및지역개발,지역및도시
2,2024,20240704,경기,경기수원시,일반회계,세류중학교 인조잔디운동장 시설 개선(도비),250000000,0,250000000,문화및관광,체육
3,2024,20240704,경기,경기의왕시,일반회계,한국철도공사 인재개발원 인조잔디구장 조성공사,1000000000,0,0,문화및관광,체육
4,2024,20240704,경기,경기이천시,수질개선특별회계,신둔면 체육공원 풋살장 및 족구장 인조잔디 설치(보조),289766000,0,289766000,문화및관광,체육
...,...,...,...,...,...,...,...,...,...,...,...
2045,2024,20240704,제주,제주본청,일반회계,서귀포추모공원 자연장지 확장 사업,1790000000,0,1790000000,사회복지,노인ㆍ청소년
2046,2024,20240704,제주,제주본청,일반회계,성산읍 테니스장 확충공사,100000000,0,100000000,문화및관광,체육
2047,2024,20240704,제주,제주본청,일반회계,장기미집행 도시계획시설(제1-55호 어린이공원) 조성사업,670000000,0,670000000,국토및지역개발,지역및도시
2048,2024,20240704,제주,제주본청,일반회계,장기미집행 도시계획시설(제1-57호 어린이공원) 조성사업,760000000,0,760000000,국토및지역개발,지역및도시


In [387]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(budget_df_today_final,'budget','budget_df_listup',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_delete,'budget','budget_df_delete',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_new,'budget','budget_df_new',bigquerykey_path)

Data inserted into table budget_df_listup successfully.
Data inserted into table budget_df_delete successfully.
Data inserted into table budget_df_new successfully.


#### 교육청 예산서

In [388]:
# 종합쇼핑몰 학교 관련 사업
shop_edu_mapping_temp1 = all_shop_df_fin[all_shop_df_fin['납품요구건명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_temp2 = all_shop_df_fin[all_shop_df_fin['수요기관명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_df = pd.concat([shop_edu_mapping_temp1,shop_edu_mapping_temp2],axis=0).drop_duplicates().reset_index(drop=True)
shop_edu_mapping_df

,납품요구접수일자,수요기관명,납품요구건명,업체명,단가,단위,수량,금액,수요기관코드,수요기관구분,수요기관지역명,납품요구지청명,납품요구건명_re
0,2024-07-03,인천광역시남부교육청 인성초등학교,인성초등학교 운동장 인조잔디 교체,케이앤비준우(주),73000,㎡,788.0,57524000,7320148,기타기관,인천광역시 중구,인천지방조달청,인성 운동장 인조잔디 교체
1,2024-07-03,서울특별시동부교육청 상봉중학교,상봉중학교 운동장 환경개선관련 관급자재 구매,주식회사 액션필드,60700,㎡,2597.0,157637900,7020148,교육기관,서울특별시 중랑구,서울지방조달청,상봉 운동장 환경개선관련 관급자재 구매
2,2024-07-03,선린대학,"선린대학교 운동장 환경개선(인조잔디, 트랙 등) 공사 관급자재 발주",주식회사 네오필드,74500,㎡,4890.0,364305000,7003548,기타기관,경상북도 포항시 북구,대구지방조달청,선린대학교 운동장 환경개선인조잔디 트랙 등 공사 관급자재 발주
3,2024-07-02,경기도용인교육청 용동중학교,용동중학교 운동장개선 공사 인조잔디 구입,케이앤비준우(주),39500,㎡,3600.0,142200000,7750018,기타기관,경기도 용인시 처인구,인천지방조달청,용동 운동장개선 공사 인조잔디 구입
4,2024-07-01,충청남도교육청 충남삼성고등학교,충남삼성고등학교 운동장 인조잔디 구매,주식회사 플랜에이,63200,㎡,3070.0,194024000,8140366,기타기관,충청남도 아산시,대전지방조달청,충남삼성 운동장 인조잔디 구매
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2024-01-09,서울특별시강동교육청 서울가동초등학교,실내운동장 인조잔디 구입,주식회사 필드글로벌,63000,㎡,495.0,31185000,7070052,교육기관,서울특별시 송파구,서울지방조달청,실내운동장 인조잔디 구입
225,2024-01-08,제주특별자치도교육청 제주시교육지원청 동광초등학교,동광초 인조잔디 및 울타리 조성공사 관급자재 구입(인조잔디 외 1종),주식회사 필드글로벌,63000,㎡,2680.0,168840000,9296060,교육기관,제주특별자치도 제주시,제주지방조달청,동광초 인조잔디 및 울타리 조성공사 관급자재 구입인조잔디 외 종
226,2024-01-04,한국과학기술원부설한국과학영재학교,풋살장 조성 및 기타공사 관급자재 구매,주식회사 이원,38000,㎡,747.0,28386000,B552516,기타공공기관,부산광역시 부산진구,부산지방조달청,풋살장 조성 및 기타공사 관급자재 구매
227,2024-01-03,경기도파주교육청 파주중학교,파주중 인조잔디 운동장 조성사업 관급자재 구입,주식회사 지에스케이,61500,㎡,2540.0,156210000,7680017,교육기관,경기도 파주시,서울지방조달청,파주중 인조잔디 운동장 조성사업 관급자재 구입


In [389]:
# 교육청, 교육지원청, 학교명 필터링
shop_edu_mapping_df['수요기관명_re'] = shop_edu_mapping_df['수요기관명'].str.replace('교육청','').str.replace('교육지원청','')
filter_edu1 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[0].notnull()]['수요기관명_re'].str.split().str[0].unique())
filter_edu2 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[1].notnull()]['수요기관명_re'].str.split().str[1].unique())
filter_edu3 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[2].notnull()]['수요기관명_re'].str.split().str[2].unique())

In [390]:
# 전체 키워드 데이터셋
edu_mapping_keywd_all = list(chain(*shop_edu_mapping_df['납품요구건명_re'].str.split(' '))) ## 전체 split 키워드
edu_mapping_keywd_df = pd.DataFrame(Counter(edu_mapping_keywd_all).items(),columns=['키워드','빈도수'])
edu_mapping_keywd_df = edu_mapping_keywd_df[edu_mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
edu_mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
2,인조잔디,130
1,운동장,84
15,구입,77
7,구매,64
55,관급자재인조잔디,63
...,...,...
164,토목공사,1
162,화산,1
161,목포부주,1
160,인조잔디구매,1


In [391]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,"89,533,000","4,664",학교
1,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
2,강원특별자치도,강원특별자치도교육청,공립,강릉제일고등학교 축구장 인조잔디 교체,"1,020,560,000","8,576",교육청
3,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,"806,400,000","4,082",교육청
4,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
...,...,...,...,...,...,...,...
189,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None
190,충청남도,당진교육지원청,공립,면천중학교 운동장 교체,"297,561,000",None,None
191,충청북도,충청북도교육청,공립,충북산업과학고등학교 다목적구장 조성,"769,799,000","5,540",교육청
192,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,"602,871,000","3,500",교육청


In [392]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [394]:
# 종합쇼핑몰 형식과 통일
edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [396]:
# 교육청, 교육지원청, 학교명 추출
filter_edu1_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu1)],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu1))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu1))]],axis=0)
# edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu1))].shape

filter_edu2_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu2)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu2))]],axis=0)

filter_edu3_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu3)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu3))]],axis=0)

filtered_edu_budget_df = pd.concat([filter_edu1_temp, filter_edu2_temp, filter_edu3_temp],axis=0).drop_duplicates().reset_index(drop=True)

In [406]:
# 종합쇼핑몰 학교 관련 키워드 빈도수
edu_filter_keywd = ('|').join(edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3]['키워드'].unique())
edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3].sort_values('빈도수',ascending=False) ## 빈도수 평균 2.99

,키워드,빈도수
43,다목적,3
25,운동장환경개선,3
3,교체,3
117,따른,3
47,물품,3
...,...,...
162,화산,1
161,목포부주,1
160,인조잔디구매,1
159,동북,1


In [420]:
# 교육청 제외 데이터
filtered_edu_budget_df_fin = filtered_edu_budget_df[filtered_edu_budget_df['과업명'].str.contains(edu_filter_keywd)].drop_duplicates().reset_index(drop=True)
filtered_edu_budget_df_fin

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None,성주 운동장 교체,충청남도보령
1,충청남도,보령교육지원청,공립,청파초등학교 운동장 교체,"20,976,000",None,None,청파 운동장 교체,충청남도보령
2,경기도,김포교육지원청,공립,신양중학교 친환경 운동장 조성 사업,None,None,None,신양 친환경 운동장 조성 사업,경기도김포
3,경기도,수원교육지원청,공립,수성중학교 친환경 운동장 조성 사업,None,None,None,수성 친환경 운동장 조성 사업,경기도수원
4,경기도,연천교육지원청,공립,백학중학교 친환경 운동장 조성 사업,None,None,None,백학 친환경 운동장 조성 사업,경기도연천
5,경기도,용인교육지원청,공립,어정초등학교 친환경 운동장 조성 사업,None,None,None,어정 친환경 운동장 조성 사업,경기도용인
6,경기도,용인교육지원청,공립,서원중학교 친환경 운동장 조성 사업,None,None,None,서원 친환경 운동장 조성 사업,경기도용인
7,경기도,파주교육지원청,공립,와석초등학교 친환경 운동장 조성 사업,None,None,None,와석 친환경 운동장 조성 사업,경기도파주
8,경기도,포천교육지원청,공립,영북초등학교 친환경 운동장 조성 사업,None,None,None,영북 친환경 운동장 조성 사업,경기도포천
9,서울특별시,동부교육지원청,공립,상봉중학교(운동장 환경 개선),"680,000,000","4,703",학교,상봉운동장 환경 개선,서울특별시동부


In [418]:
edu_budget_df[~edu_budget_df['과업명'].isin(filtered_edu_budget_df_fin['과업명'])].reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,전북특별자치도,전북특별자치도교육청,None,완주 스포츠클럽 실외야구장,"1,689,820,000",None,None,완주 스포츠클럽 실외야구장,전북특별자치도교육청
1,경기도,경기도교육청,None,친환경 운동장 조성사업비 지원 (30교),"15,000,000,000",None,학교,친환경 운동장 조성사업비 지원 교,경기도교육청
2,전라남도,전라남도교육청,None,운동장 개보수(고2),"1,652,038,000",None,학교,운동장 개보수고,전라남도교육청
3,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 중학교 1교,"14,000,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
4,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 고등학교 6교,"465,600,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
...,...,...,...,...,...,...,...,...,...
138,경상남도,의령교육지원청,사립,정곡중학교 체육관 신축(계속비),"38,000,000",None,학교,정곡 체육관 신축계속비,경상남도의령
139,대구광역시,대구광역시교육청,사립,대륜고등학교 축구장 인조잔디 재조성,"578,358,000","7,447",학교,대륜 축구장 인조잔디 재조성,대구광역시교육청
140,울산광역시,울산광역시교육청,사립,친환경운동장조성 다목적구장 중학교 (사립) 1교,"169,141,500",None,학교,친환경운동장조성 다목적구장 사립 교,울산광역시교육청
141,전북특별자치도,전북특별자치도교육청,사립,익산고등학교 인조잔디 조성,"1,050,001,000","7,417",교육청,익산 인조잔디 조성,전북특별자치도교육청
